In [ ]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.graph_objects as go

In [ ]:
with open('../data/nash_hosp_referrers.pickle', 'rb') as file:
    nash_hosp_referrers = pickle.load(file)

In [ ]:
with open('../data/nash_hosp_referrers_b.pickle', 'rb') as file:
    nash_hosp_referrers_b = pickle.load(file)

In [ ]:
nash_hosp_referrers_b

In [ ]:
@interact(specialty = nash_hosp_referrers_b['specialty'].unique())
def interactive_plot(specialty):
    
    df = (nash_hosp_referrers_b
          .loc[(~nash_hosp_referrers_b['hospital'].str.contains('Vanderbilt'))
               &
               (nash_hosp_referrers_b['specialty'] == specialty)]
          .groupby(['from_npi', 'location_address_state_name', 'location_address_city_name', 'specialty'])
          ['transaction_count'].sum()
          .to_frame()
          .reset_index())
    
    fig = px.treemap(df.loc[df['transaction_count'] > 100], 
                     path=[px.Constant("all"), 'location_address_state_name', 'specialty', 'from_npi'], 
                     values='transaction_count',
                     labels = {'location_address_state_name' : 'State',
                                  #'id' : False,
                                  'transaction_count' : 'Total non-Vanderbilt referrals'})

    fig.show()

In [ ]:
@interact(specialty = np.sort(nash_hosp_referrers_b['specialty'].unique()))
def interactive_plot(specialty):
    
    df = (nash_hosp_referrers_b
          .loc[(~nash_hosp_referrers_b['hospital'].str.contains('Vanderbilt'))
               &
               (nash_hosp_referrers_b['specialty'] == specialty)]
          .groupby(['from_npi', 
                    'location_address_state_name', 
                    'location_address_city_name', 
                    'specialty',
                    'sub_specialty'])
          ['transaction_count'].sum()
          .to_frame()
          .reset_index()
          .sort_values('transaction_count', ascending = False))
    
    fig = go.Figure(data=[go.Table(
        header=dict(values= ['Provider NPI', 'Specialty', 'Sub-Specialty', 'State', 'Non-Vandy Referrals'],
                    fill_color='#f4c0c5',
                    align='left'),
        cells=dict(values=[df.from_npi, 
                           df.specialty,
                           df.sub_specialty, 
                           df.location_address_state_name,
                           df.transaction_count],
                   fill_color='#fdf2f3',
                   align='left'))
    ])

    fig.show()